In [1]:
# import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf
import numpy as np
from sqlalchemy import create_engine
import psycopg2

In [2]:
# engine = psycopg2.connect(
#     database="realestate",
#     user="postgres",
#     password="password",
#     host="realestate.cpolm7b27cpl.us-east-2.rds.amazonaws.com",
#     port="5432")

In [3]:
conn = psycopg2.connect(host="realestate.cpolm7b27cpl.us-east-2.rds.amazonaws.com", port = 5432, database="realestate", user="postgres", password="password")

In [4]:
cur = conn.cursor()

In [5]:
cur.execute("""SELECT * FROM total""")

In [6]:
query_results = cur.fetchall()

In [7]:
test_df = pd.DataFrame(query_results)

In [8]:
test_df

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,5085010022,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,...,2,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
1,5085033008,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,...,1,0,0,0,0,0,0,YES,0,0
2,5085032022,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,...,1,0,0,0,0,0,0,YES,0,0
3,5510005006,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,...,5,0,1,0,0,YES,0,CENTRAL,0,0
4,5089024002,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,...,1,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6725,4335003015,La Peer Drive llc,No,Yes,115 N La Peer Dr,Los Angeles,90048,141 E Jefferson Blvd,Yes,Los Angeles,...,0,0,0,0,0,0,0,0,0,0
6726,4335010018,Cobra Acquisitions llc,No,Yes,123 S La Peer Dr,Los Angeles,90048,11301 W Olympic Blvd #206,Yes,Los Angeles,...,0,0,0,0,0,0,0,0,0,0
6727,5510018012,Aron B Bick Bar Kochba Trust,Yes,No,6217 Lindenhurst Ave,Los Angeles,90048,6217 Lindenhurst Ave,No,Los Angeles,...,0,0,0,0,YES,0,YES,FORCED,0,0
6728,5510017023,Robs Beverly Grove llc,No,Yes,6360 Maryland Dr,Los Angeles,90048,2315 Cedar Swamp Rd,Yes,Glen Head,...,0,0,0,YES,0,0,0,0,0,0


In [9]:
Beverly_Grove_Total_df = test_df

In [14]:
Beverly_Grove_Total_df[0] = Beverly_Grove_Total_df[0].astype(int)

In [15]:
Beverly_Grove_Total_df[0].dtype

dtype('int64')

In [16]:
# change the index to to be the first column
Beverly_Grove_Total_df = Beverly_Grove_Total_df.rename(columns={0:'PARCEL'})
Beverly_Grove_Total_df.set_index('PARCEL', inplace=True)
Beverly_Grove_Total_df.head()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,CA,...,2,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
5085033008,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,CA,...,1,0,0,0,0,0,0,YES,0,0
5085032022,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,CA,...,1,0,0,0,0,0,0,YES,0,0
5510005006,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,CA,...,5,0,1,0,0,YES,0,CENTRAL,0,0
5089024002,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,CA,...,1,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0


In [17]:
Beverly_Grove_Total_df.dropna()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,CA,...,2,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
5085033008,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,CA,...,1,0,0,0,0,0,0,YES,0,0
5085032022,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,CA,...,1,0,0,0,0,0,0,YES,0,0
5510005006,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,CA,...,5,0,1,0,0,YES,0,CENTRAL,0,0
5089024002,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,CA,...,1,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335003015,La Peer Drive llc,No,Yes,115 N La Peer Dr,Los Angeles,90048,141 E Jefferson Blvd,Yes,Los Angeles,CA,...,0,0,0,0,0,0,0,0,0,0
4335010018,Cobra Acquisitions llc,No,Yes,123 S La Peer Dr,Los Angeles,90048,11301 W Olympic Blvd #206,Yes,Los Angeles,CA,...,0,0,0,0,0,0,0,0,0,0
5510018012,Aron B Bick Bar Kochba Trust,Yes,No,6217 Lindenhurst Ave,Los Angeles,90048,6217 Lindenhurst Ave,No,Los Angeles,CA,...,0,0,0,0,YES,0,YES,FORCED,0,0


In [18]:
new_column_names = ['Composite Name',
'Owned by Trust?',
'Owned by Business?',
'SITEADDRESS',
'SITECITY',
'SITEZIP',
'MAILADDRESS',
'MAIL DIFFERENT FROM SITE?',
'MAILCITY',
'MAILSTATE',
'MAIL OUTSIDE CA?',
'TITLECO1',
'ASSDTOTAL',
'ASSDLAND',
'ASSDSTCT',
'ASSDOTHR',
'EXEMPTCD',
'EXMPTAMT',
'PCNTIMPD',
'TAXAMT',
'DOCDATE_YEAR',
'DEEDTYPE',
'MULTIPARCEL',
'PRICE',
'LENDER1',
'LOANAMOUT1',
'LOANTYPE1',
'YEARBLT',
'EFFYRBLT',
'LOTSQFT',
'TOTALSF',
'GRGTYPE',
'BLDGMAT',
'TOTUNITS',
'QUALCLAS',
'BLDGCOND',
'NOSTORY',
'ROOFMAT',
'FOUNDATN',
'BEDROOMS',
'BATHROOMS',
'FAMILYRM',
'DININGRM',
'POOL',
'PATIO',
'FIREPLCE',
'AIRMTHOD',
'HEATMTHD',
'WALLHGT',
'VIEW']

In [19]:
Beverly_Grove_Total_df.columns = new_column_names

In [20]:
Beverly_Grove_Total_df.head()

,Composite Name,Owned by Trust?,Owned by Business?,SITEADDRESS,SITECITY,SITEZIP,MAILADDRESS,MAIL DIFFERENT FROM SITE?,MAILCITY,MAILSTATE,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,WALLHGT,VIEW
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,CA,...,2,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
5085033008,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,CA,...,1,0,0,0,0,0,0,YES,0,0
5085032022,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,CA,...,1,0,0,0,0,0,0,YES,0,0
5510005006,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,CA,...,5,0,1,0,0,YES,0,CENTRAL,0,0
5089024002,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,CA,...,1,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0


## Now import our second file, the sales one

In [21]:
conn = psycopg2.connect(host="localhost", port = 5432, database="beverly_grove_sales", user="postgres", password="mTM36[TU")

In [22]:
cur = conn.cursor()

In [23]:
cur.execute("""SELECT * FROM beverly_grove_sales""")

In [24]:
query_results2 = cur.fetchall()

In [25]:
Beverly_Grove_Sales_df = pd.DataFrame(query_results2)
Beverly_Grove_Sales_df.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,5089002004,Astrid Meghrigian,Astrid,Meghrigian,None,NaN,None,None,Los Angeles,CA,...,6100.0,NaN,N,None,5691.0,2036,Parking Lot,2151.021000,NaN,Lac2
1,5086013015,6000 Sv Holdings Llc,None,6000 Sv Holdings Llc,None,NaN,None,None,Los Angeles,CA,...,5000.0,20000000.0,N,None,9721.0,2036,Parking Lot,2169.003001,1970.0,Lac2
2,5512016010,Scott Strumwasser & Mahtash Rahbar,Scott,Strumwasser,5971 W 3Rd St,5971.0,None,3Rd St,Los Angeles,CA,...,9487.0,NaN,N,None,7372.0,2001,"Retail Stores ( Personal Services, Photography...",2140.002009,1929.0,Lac1
3,5511028030,Hart Third Street Llc,None,Hart Third Street Llc,8038 W 3Rd St,8038.0,None,3Rd St,Los Angeles,CA,...,2205.0,NaN,N,None,7555.0,2001,"Retail Stores ( Personal Services, Photography...",2146.002001,1946.0,Lac2
4,5511029032,Triple Heights Llc,None,Triple Heights Llc,8124 W 3Rd St,8124.0,None,3Rd St,Los Angeles,CA,...,6400.0,8700000.0,N,None,7555.0,3003,Office Bldg (General),2146.002002,1967.0,Lac2


In [27]:
Beverly_Grove_Sales_df[0].dtype

dtype('int64')

In [28]:
# change the index to to be the first column

Beverly_Grove_Sales_df = Beverly_Grove_Sales_df.rename(columns={0:'PARCEL'})
Beverly_Grove_Sales_df.set_index('PARCEL', inplace=True)
Beverly_Grove_Sales_df.head()

,1,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,32,33
PARCEL,,,,,,,,,,,,,,,,,,,,,
5089002004,Astrid Meghrigian,Astrid,Meghrigian,None,NaN,None,None,Los Angeles,CA,90036,...,6100.0,NaN,N,None,5691.0,2036,Parking Lot,2151.021000,NaN,Lac2
5086013015,6000 Sv Holdings Llc,None,6000 Sv Holdings Llc,None,NaN,None,None,Los Angeles,CA,90036,...,5000.0,20000000.0,N,None,9721.0,2036,Parking Lot,2169.003001,1970.0,Lac2
5512016010,Scott Strumwasser & Mahtash Rahbar,Scott,Strumwasser,5971 W 3Rd St,5971.0,None,3Rd St,Los Angeles,CA,90036,...,9487.0,NaN,N,None,7372.0,2001,"Retail Stores ( Personal Services, Photography...",2140.002009,1929.0,Lac1
5511028030,Hart Third Street Llc,None,Hart Third Street Llc,8038 W 3Rd St,8038.0,None,3Rd St,Los Angeles,CA,90048,...,2205.0,NaN,N,None,7555.0,2001,"Retail Stores ( Personal Services, Photography...",2146.002001,1946.0,Lac2
5511029032,Triple Heights Llc,None,Triple Heights Llc,8124 W 3Rd St,8124.0,None,3Rd St,Los Angeles,CA,90048,...,6400.0,8700000.0,N,None,7555.0,3003,Office Bldg (General),2146.002002,1967.0,Lac2


In [29]:
column_names = ['Owner Name2', 'Owner1 First Name', 'Owner1 Last Name', 
                'Site Address', 'Site Address House Number', 'Site Address Unit Number', 
                'Site Address Street Name', 'Site Address City', 'Site Address State', 
                'Site Address Zip', 'Mail Address', 'Mail Address Unit Number', 
                'Mail Address City', 'Mail Address State', 'Mail Address Zip', 
                'Number of Units', 'Owner Occupied', 'Bathrooms', 'Pool', 'Bedrooms', 
                'Sale Date', 'Lot Area Acres', 'Lot Area SQFT', 'Building Area', 
                'Sales Price', 'Tax Delinquent', 'Tax Exemption Code', 'Tract', 
                'Use Code', 'Use Code Description', 'Census Tract', 'Year Built', 'Zoning']

In [30]:
Beverly_Grove_Sales_df.columns = column_names

In [31]:
Beverly_Grove_Sales_df.head()

,Owner Name2,Owner1 First Name,Owner1 Last Name,Site Address,Site Address House Number,Site Address Unit Number,Site Address Street Name,Site Address City,Site Address State,Site Address Zip,...,Building Area,Sales Price,Tax Delinquent,Tax Exemption Code,Tract,Use Code,Use Code Description,Census Tract,Year Built,Zoning
PARCEL,,,,,,,,,,,,,,,,,,,,,
5089002004,Astrid Meghrigian,Astrid,Meghrigian,None,NaN,None,None,Los Angeles,CA,90036,...,6100.0,NaN,N,None,5691.0,2036,Parking Lot,2151.021000,NaN,Lac2
5086013015,6000 Sv Holdings Llc,None,6000 Sv Holdings Llc,None,NaN,None,None,Los Angeles,CA,90036,...,5000.0,20000000.0,N,None,9721.0,2036,Parking Lot,2169.003001,1970.0,Lac2
5512016010,Scott Strumwasser & Mahtash Rahbar,Scott,Strumwasser,5971 W 3Rd St,5971.0,None,3Rd St,Los Angeles,CA,90036,...,9487.0,NaN,N,None,7372.0,2001,"Retail Stores ( Personal Services, Photography...",2140.002009,1929.0,Lac1
5511028030,Hart Third Street Llc,None,Hart Third Street Llc,8038 W 3Rd St,8038.0,None,3Rd St,Los Angeles,CA,90048,...,2205.0,NaN,N,None,7555.0,2001,"Retail Stores ( Personal Services, Photography...",2146.002001,1946.0,Lac2
5511029032,Triple Heights Llc,None,Triple Heights Llc,8124 W 3Rd St,8124.0,None,3Rd St,Los Angeles,CA,90048,...,6400.0,8700000.0,N,None,7555.0,3003,Office Bldg (General),2146.002002,1967.0,Lac2


In [32]:
# create a new column which will later be transformed into our output column after we've merged the two df's

Beverly_Grove_Sales_df['Did it sell?'] = 'yes'

# check the new column
Beverly_Grove_Sales_df.head()

,Owner Name2,Owner1 First Name,Owner1 Last Name,Site Address,Site Address House Number,Site Address Unit Number,Site Address Street Name,Site Address City,Site Address State,Site Address Zip,...,Sales Price,Tax Delinquent,Tax Exemption Code,Tract,Use Code,Use Code Description,Census Tract,Year Built,Zoning,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5089002004,Astrid Meghrigian,Astrid,Meghrigian,None,NaN,None,None,Los Angeles,CA,90036,...,NaN,N,None,5691.0,2036,Parking Lot,2151.021000,NaN,Lac2,yes
5086013015,6000 Sv Holdings Llc,None,6000 Sv Holdings Llc,None,NaN,None,None,Los Angeles,CA,90036,...,20000000.0,N,None,9721.0,2036,Parking Lot,2169.003001,1970.0,Lac2,yes
5512016010,Scott Strumwasser & Mahtash Rahbar,Scott,Strumwasser,5971 W 3Rd St,5971.0,None,3Rd St,Los Angeles,CA,90036,...,NaN,N,None,7372.0,2001,"Retail Stores ( Personal Services, Photography...",2140.002009,1929.0,Lac1,yes
5511028030,Hart Third Street Llc,None,Hart Third Street Llc,8038 W 3Rd St,8038.0,None,3Rd St,Los Angeles,CA,90048,...,NaN,N,None,7555.0,2001,"Retail Stores ( Personal Services, Photography...",2146.002001,1946.0,Lac2,yes
5511029032,Triple Heights Llc,None,Triple Heights Llc,8124 W 3Rd St,8124.0,None,3Rd St,Los Angeles,CA,90048,...,8700000.0,N,None,7555.0,3003,Office Bldg (General),2146.002002,1967.0,Lac2,yes


In [33]:
# drop the useless columns, i.e only keep 'PARCEL' and 'Did it sell?'

Beverly_Grove_Sales_df = Beverly_Grove_Sales_df[['Did it sell?']]

In [34]:
# check to see if it worked.

Beverly_Grove_Sales_df.head()

,Did it sell?
PARCEL,
5089002004,yes
5086013015,yes
5512016010,yes
5511028030,yes
5511029032,yes


## Merge our dataframes

In [35]:
Beverly_Grove_Total_df = Beverly_Grove_Total_df.merge(Beverly_Grove_Sales_df, how='left', left_index=True, right_index=True)
Beverly_Grove_Total_df.head()

,Composite Name,Owned by Trust?,Owned by Business?,SITEADDRESS,SITECITY,SITEZIP,MAILADDRESS,MAIL DIFFERENT FROM SITE?,MAILCITY,MAILSTATE,...,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,WALLHGT,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,CA,...,0,1,0,0,YES,CENTRAL,CENTRAL,0,0,NaN
5085033008,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,CA,...,0,0,0,0,0,0,YES,0,0,NaN
5085032022,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,CA,...,0,0,0,0,0,0,YES,0,0,NaN
5510005006,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,CA,...,0,1,0,0,YES,0,CENTRAL,0,0,NaN
5089024002,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,CA,...,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0,NaN


In [36]:
# Fill in the NaN's in the 'Did it sell' column
Beverly_Grove_Total_df['Did it sell?'] = Beverly_Grove_Total_df['Did it sell?'].fillna('no')

In [37]:
# check the value counts
Beverly_Grove_Total_df['Did it sell?'].value_counts()

no     6228
yes     502
Name: Did it sell?, dtype: int64

## Drop the columns we don't need

In [38]:
Beverly_Grove_Total_df.columns

Index(['Composite Name', 'Owned by Trust?', 'Owned by Business?',
       'SITEADDRESS', 'SITECITY', 'SITEZIP', 'MAILADDRESS',
       'MAIL DIFFERENT FROM SITE?', 'MAILCITY', 'MAILSTATE',
       'MAIL OUTSIDE CA?', 'TITLECO1', 'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT',
       'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT', 'PCNTIMPD', 'TAXAMT',
       'DOCDATE_YEAR', 'DEEDTYPE', 'MULTIPARCEL', 'PRICE', 'LENDER1',
       'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF',
       'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY',
       'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM',
       'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'WALLHGT', 'VIEW',
       'Did it sell?'],
      dtype='object')

In [39]:
# drop 'Composite Name', 'SITEADDRESS', 'SITECITY', 'SITEZIP', 'MAILADDRESS','WALLHGT'
Beverly_Grove_Total_df.drop(['Composite Name', 'SITEADDRESS', 'SITECITY', 'SITEZIP',
                             'MAILADDRESS', 'WALLHGT', 'MAILCITY', 'MAILSTATE'],
                              axis=1, inplace=True)

Beverly_Grove_Total_df.head()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,No,No,No,No,FIRST AMERICAN TITLE,1157385,925908,231477,0,0,...,2,0,1,0,0,YES,CENTRAL,CENTRAL,0,no
5085033008,No,No,Yes,No,0,57047,40994,16053,0,0,...,1,0,0,0,0,0,0,YES,0,no
5085032022,No,No,No,No,ATTORNEY ONLY,651485,521193,130292,0,HOMEOWNERS,...,1,0,0,0,0,0,0,YES,0,no
5510005006,Yes,No,Yes,No,0,85697,74539,11158,0,0,...,5,0,1,0,0,YES,0,CENTRAL,0,no
5089024002,No,No,No,No,0,62487,42013,20474,0,HOMEOWNERS,...,1,1,0,0,YES,YES,CENTRAL,CENTRAL,0,no


## Encode Column Values

In [40]:
# Determine the number of unique values in each column.

Beverly_Grove_Total_df.apply(lambda x: len(x.unique()))

Owned by Trust?                 2
Owned by Business?              2
MAIL DIFFERENT FROM SITE?       2
MAIL OUTSIDE CA?                2
TITLECO1                      119
ASSDTOTAL                    6137
ASSDLAND                     5124
ASSDSTCT                     5603
ASSDOTHR                        5
EXEMPTCD                        2
EXMPTAMT                       14
PCNTIMPD                       94
TAXAMT                       6671
DOCDATE_YEAR                   58
DEEDTYPE                       23
MULTIPARCEL                     3
PRICE                        1172
LENDER1                       504
LOANAMOUT1                   1394
LOANTYPE1                      11
YEARBLT                        98
EFFYRBLT                       99
LOTSQFT                      1757
TOTALSF                      2186
GRGTYPE                         7
BLDGMAT                         9
TOTUNITS                       34
QUALCLAS                        6
BLDGCOND                        6
NOSTORY       

In [41]:
# Convert binary value counts to 0's and 1's, since most of them are 'Yes' or 'No' right now.
# Step 1: 'Owned by Trust?'

Beverly_Grove_Total_df['Owned by Trust?'].replace({"Yes": 1, "No": 0}, inplace=True)

In [42]:
# Step 2: 'Owned by Business?'
Beverly_Grove_Total_df['Owned by Business?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 3: 'MAIL DIFFERENT FROM SITE?'
Beverly_Grove_Total_df['MAIL DIFFERENT FROM SITE?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 4: 'MAIL OUTSIDE CA'
Beverly_Grove_Total_df['MAIL OUTSIDE CA?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 5: 'EXEMPTCD'
Beverly_Grove_Total_df['EXEMPTCD'].replace({"HOMEOWNERS": 1}, inplace=True)

# Step 6: 'Did it sell?'
Beverly_Grove_Total_df['Did it sell?'].replace({"yes": 1, "no": 0}, inplace=True)

In [43]:
Beverly_Grove_Total_df.head()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,FIRST AMERICAN TITLE,1157385,925908,231477,0,0,...,2,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,1,0,0,0,0,0,0,YES,0,0
5085032022,0,0,0,0,ATTORNEY ONLY,651485,521193,130292,0,1,...,1,0,0,0,0,0,0,YES,0,0
5510005006,1,0,1,0,0,85697,74539,11158,0,0,...,5,0,1,0,0,YES,0,CENTRAL,0,0
5089024002,0,0,0,0,0,62487,42013,20474,0,1,...,1,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0


In [44]:
Beverly_Grove_Total_df['Owned by Trust?'].value_counts()

0    5033
1    1697
Name: Owned by Trust?, dtype: int64

In [45]:
Beverly_Grove_Total_df['Owned by Trust?'].dtypes

dtype('int64')

In [46]:
Beverly_Grove_Total_df['FAMILYRM'].value_counts()

0    6443
1     284
2       3
Name: FAMILYRM, dtype: int64

## Now encode the categorical data

In [47]:
Beverly_Grove_Total_df.columns

Index(['Owned by Trust?', 'Owned by Business?', 'MAIL DIFFERENT FROM SITE?',
       'MAIL OUTSIDE CA?', 'TITLECO1', 'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT',
       'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT', 'PCNTIMPD', 'TAXAMT',
       'DOCDATE_YEAR', 'DEEDTYPE', 'MULTIPARCEL', 'PRICE', 'LENDER1',
       'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF',
       'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY',
       'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM',
       'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'VIEW',
       'Did it sell?'],
      dtype='object')

In [48]:
Encoded_List = [
    'TITLECO1',
    'MULTIPARCEL',
    'LENDER1',
    'LOANTYPE1',
    'GRGTYPE',
    'BLDGMAT',
    'QUALCLAS',
    'BLDGCOND',
    'POOL',
    'PATIO',
    'FIREPLCE',
    'AIRMTHOD',
    'HEATMTHD',
    'VIEW',
    'ROOFMAT',
    'FOUNDATN',
]

In [49]:
for col in Beverly_Grove_Total_df:
    if col in Encoded_List:
        Beverly_Grove_Total_df[col] = Beverly_Grove_Total_df[col].transform(lambda x: x != '0')
        Beverly_Grove_Total_df[col] = Beverly_Grove_Total_df[col].astype(int)

In [50]:
Beverly_Grove_Total_df.head()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,2,0,1,0,0,1,1,1,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,1,0,0,0,0,0,0,1,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,1,0,0,0,0,0,0,1,0,0
5510005006,1,0,1,0,0,85697,74539,11158,0,0,...,5,0,1,0,0,1,0,1,0,0
5089024002,0,0,0,0,0,62487,42013,20474,0,1,...,1,1,0,0,1,1,1,1,0,0


In [51]:
Beverly_Grove_Total_df.columns

Index(['Owned by Trust?', 'Owned by Business?', 'MAIL DIFFERENT FROM SITE?',
       'MAIL OUTSIDE CA?', 'TITLECO1', 'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT',
       'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT', 'PCNTIMPD', 'TAXAMT',
       'DOCDATE_YEAR', 'DEEDTYPE', 'MULTIPARCEL', 'PRICE', 'LENDER1',
       'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF',
       'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY',
       'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM',
       'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'VIEW',
       'Did it sell?'],
      dtype='object')

In [53]:
Beverly_Grove_Total_df.dtypes

Owned by Trust?                int64
Owned by Business?             int64
MAIL DIFFERENT FROM SITE?      int64
MAIL OUTSIDE CA?               int64
TITLECO1                       int64
ASSDTOTAL                      int64
ASSDLAND                       int64
ASSDSTCT                       int64
ASSDOTHR                       int64
EXEMPTCD                      object
EXMPTAMT                       int64
PCNTIMPD                       int64
TAXAMT                       float64
DOCDATE_YEAR                   int64
DEEDTYPE                      object
MULTIPARCEL                    int64
PRICE                          int64
LENDER1                        int64
LOANAMOUT1                     int64
LOANTYPE1                      int64
YEARBLT                        int64
EFFYRBLT                     float64
LOTSQFT                        int64
TOTALSF                        int64
GRGTYPE                        int64
BLDGMAT                        int64
TOTUNITS                       int64
Q

## Now encode 'Deedtype'

In [54]:
Deedtype_Series = Beverly_Grove_Total_df.groupby('DEEDTYPE').count()['ASSDTOTAL'].tail(-10).index
Deedtype_Series

Index(['GRANT DEED', 'IN LIEU OF FORC', 'INTERSPOUSAL', 'JOINT TENANT',
       'NAME CHANGE', 'PARTNERSHIP', 'PERSONAL REP', 'QUIT CLAIM', 'RE-RECORD',
       'TAX DEED', 'TRUST TRANSFER', 'TRUSTEES', 'WARRANTY'],
      dtype='object', name='DEEDTYPE')

In [55]:
Beverly_Grove_Total_df = pd.get_dummies(Beverly_Grove_Total_df, columns=['DEEDTYPE'], prefix="", prefix_sep="")

In [56]:
Beverly_Grove_Total_df.sample(5)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,JOINT TENANT,NAME CHANGE,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5508005012,0,0,0,0,1,648556,518847,129709,0,1,...,0,0,0,0,0,0,0,0,0,0
5525036005,1,0,1,0,1,96672,58587,38085,0,0,...,0,0,0,0,0,0,0,0,0,0
5511020011,0,0,0,0,1,988675,686765,301910,0,0,...,0,0,0,0,0,0,0,0,0,0
5512022006,1,0,0,0,1,927348,741889,185459,0,1,...,0,0,0,0,1,0,0,0,0,0
5514003043,0,0,0,0,1,2284312,1421350,862962,0,0,...,0,0,0,0,1,0,0,0,0,0


In [57]:

Beverly_Grove_Total_df['QUIT CLAIM'].value_counts()

0    5753
1     977
Name: QUIT CLAIM, dtype: int64

In [58]:
Beverly_Grove_Total_df['Did it sell?'].value_counts()

0    6228
1     502
Name: Did it sell?, dtype: int64

In [59]:
# drop any NaN's
Beverly_Grove_Total_df = Beverly_Grove_Total_df.dropna()
Beverly_Grove_Total_df

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,JOINT TENANT,NAME CHANGE,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,1,0,0,85697,74539,11158,0,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,0,0,0,62487,42013,20474,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335003015,0,1,1,0,0,1177690,1177690,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4335010018,0,1,1,0,1,1346195,1346195,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5510018012,1,0,0,0,0,762455,509154,253301,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
Beverly_Grove_Total_df['EXEMPTCD'] = Beverly_Grove_Total_df['EXEMPTCD'].astype(int)

In [61]:
Beverly_Grove_Total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6725 entries, 5085010022 to 5514001029
Data columns (total 65 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Owned by Trust?            6725 non-null   int64  
 1   Owned by Business?         6725 non-null   int64  
 2   MAIL DIFFERENT FROM SITE?  6725 non-null   int64  
 3   MAIL OUTSIDE CA?           6725 non-null   int64  
 4   TITLECO1                   6725 non-null   int64  
 5   ASSDTOTAL                  6725 non-null   int64  
 6   ASSDLAND                   6725 non-null   int64  
 7   ASSDSTCT                   6725 non-null   int64  
 8   ASSDOTHR                   6725 non-null   int64  
 9   EXEMPTCD                   6725 non-null   int64  
 10  EXMPTAMT                   6725 non-null   int64  
 11  PCNTIMPD                   6725 non-null   int64  
 12  TAXAMT                     6725 non-null   float64
 13  DOCDATE_YEAR               6725 n

## Create our X and y values

In [62]:
# remove the values
y = Beverly_Grove_Total_df['Did it sell?']
X = Beverly_Grove_Total_df.drop(['Did it sell?'], 1)

In [63]:
APN = Beverly_Grove_Total_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

## Split our data into a training and test set

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [65]:
X_train

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,JOINT TENANT,NAME CHANGE,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5088004037,1,0,0,0,0,368000,147000,221000,0,1,...,0,0,0,0,0,0,0,1,0,0
5511023012,0,0,0,0,1,2193267,1370587,822680,0,0,...,0,0,0,0,0,0,0,0,0,0
4334015112,0,0,0,0,1,681232,399602,281630,0,0,...,0,0,0,0,0,0,0,0,0,0
5512023034,1,0,0,0,1,532312,310648,221664,0,1,...,0,0,0,0,0,0,0,0,0,0
5086012019,0,0,0,0,1,1114089,808511,305578,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4333021077,0,0,0,0,1,573615,371276,202339,0,0,...,0,0,0,0,0,0,0,0,0,0
4334019002,0,0,0,0,1,1189265,951415,237850,0,1,...,0,0,0,0,0,0,0,0,0,0
5525028025,0,0,1,0,0,419551,93776,325775,0,0,...,0,0,0,0,0,0,0,0,0,0


## Scale our data

In [66]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [67]:
X_train_scaled_APN = []

In [68]:
for index, parcel in enumerate(X_train.index.values):
     X_train_scaled_APN.append(np.insert(X_train_scaled[index], 0, [parcel]))

In [69]:
# check out scaled data
X_train_scaled_APN[:2]

[array([ 5.08800404e+09,  1.73182187e+00, -1.96237257e-01, -6.13485639e-01,
        -1.62023228e-01, -1.41864189e+00, -5.56152426e-01, -7.54291760e-01,
        -1.08139113e-02, -1.62825520e-02,  1.03204194e+00,  7.58908366e-02,
         1.22220356e+00, -5.48919411e-01,  1.82944178e-01, -8.23879967e-02,
        -5.83908174e-01, -8.37849166e-01, -6.07785024e-01, -8.14406886e-01,
        -1.02176266e-01, -1.35881870e-01, -2.82627036e-01,  6.36259242e-01,
         8.25227241e-01,  7.09632171e-01, -3.63379832e-01, -6.48508582e-01,
        -6.47583136e-01,  1.91522987e+00,  7.15642319e-01,  8.13733471e-01,
         1.14193225e+00,  7.04879279e-01, -2.05886051e-01,  8.59631685e-01,
        -4.35185929e-01, -4.69861458e-01,  8.10035577e-01, -9.19728460e-01,
         7.19887485e-02, -1.50020274e-01, -3.76120628e-01, -1.37817764e-01,
        -2.43975018e-02, -1.40831103e-02, -4.22828886e-02, -5.08379095e-02,
        -5.64164232e-02, -2.81746037e-02, -1.40831103e-02, -3.98607307e-02,
        -1.2

In [70]:
X_train.shape

(5043, 64)

In [71]:
# check to see that the mean is approximately 0
print(np.mean(X_train_scaled[:,0]))

1.4089683437638315e-17


In [72]:
# check to see that the standard deviation is approximately 1
print(np.std(X_train_scaled[:,0]))

1.0


## Create a learning model

In [73]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5043, 64)
(1682, 64)
(5043,)
(1682,)


In [74]:
X_train_scaled

array([[ 1.73182187, -0.19623726, -0.61348564, ...,  3.59596152,
        -0.04457451, -0.0243975 ],
       [-0.57742659, -0.19623726, -0.61348564, ..., -0.27808974,
        -0.04457451, -0.0243975 ],
       [-0.57742659, -0.19623726, -0.61348564, ..., -0.27808974,
        -0.04457451, -0.0243975 ],
       ...,
       [-0.57742659, -0.19623726,  1.63003001, ..., -0.27808974,
        -0.04457451, -0.0243975 ],
       [-0.57742659, -0.19623726, -0.61348564, ..., -0.27808974,
        -0.04457451, -0.0243975 ],
       [-0.57742659, -0.19623726, -0.61348564, ..., -0.27808974,
        -0.04457451, -0.0243975 ]])

In [75]:
np.isnan(X_train_scaled)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [76]:
y_train

PARCEL
5088004037    0
5511023012    0
4334015112    0
5512023034    0
5086012019    0
             ..
4333021077    0
4334019002    0
5525028025    0
5526026021    1
5525021018    0
Name: Did it sell?, Length: 5043, dtype: int64

In [77]:
pd.isna(X_train_scaled[1])

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [78]:
X_train_scaled.dtype

dtype('float64')

In [79]:
print(np.isnan(X_train_scaled))

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [80]:
X_train_scaled.max()

71.00704190430692

In [81]:
y_train.min()

0

In [82]:
type(X_train_scaled)

numpy.ndarray

In [83]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=300,
                                random_state=78,
                                class_weight="balanced")

In [84]:
classifier.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [85]:
y_test.head()

PARCEL
5514009006    0
4335008212    0
4334009113    0
4335012137    0
5085007005    0
Name: Did it sell?, dtype: int64

In [86]:
Beverly_Grove_Total_df.head()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,JOINT TENANT,NAME CHANGE,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,1,0,0,85697,74539,11158,0,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,0,0,0,62487,42013,20474,0,1,...,0,0,0,0,0,0,0,0,0,0


In [87]:
y_pred = classifier.predict(X_test_scaled)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test, "PARCEL": y_test.index.values}).reset_index(drop=True)
results.sample(20)

,Prediction,Actual,PARCEL
12,1,0,5526028019
1502,1,0,5527026002
421,1,1,5089017030
1439,1,0,5526037027
752,1,0,5087002015
1346,0,0,5087008014
365,0,0,5085009003
563,0,0,5527033022
1179,1,0,4335005051
1195,0,0,5088003045


In [88]:
len(results)

1682

In [89]:
Beverly_Grove_Total_Results_df = Beverly_Grove_Total_df.merge(results, how="left", right_on="PARCEL", left_on="PARCEL")
Beverly_Grove_Total_Results_df.head()

,PARCEL,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY,Prediction,Actual
0,5085010022,0,0,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,5085033008,0,0,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0.0,0.0
2,5085032022,0,0,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,5510005006,1,0,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,5089024002,0,0,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [90]:
X_test_scaled[33]

array([-0.57742659, -0.19623726, -0.61348564, -0.16202323, -1.41864189,
        1.67256486,  0.90773231,  2.69736695, -0.01628255, -0.96895287,
       -0.13491545,  0.58746188,  1.66682244,  0.18294418, -0.082388  ,
       -0.58390817,  1.19353225, -0.60778502,  1.22788746, -0.10217627,
        0.18352419, -0.36800308,  1.40351208,  0.82522724,  0.70963217,
       -0.36337983, -0.64850858, -0.64758314,  0.34678163,  0.71564232,
        0.81373347,  0.19383144,  1.71330657, -0.20588605,  0.85963169,
       -0.43518593, -0.46986146,  0.81003558,  1.08727743,  0.07198875,
       -0.15002027, -0.37612063, -0.13781776, -0.0243975 , -0.01408311,
       -0.04228289, -0.05083791, -0.05641642, -0.0281746 , -0.01408311,
       -0.03986073, -1.25104229,  0.        , -0.13781776, -0.0281746 ,
       -0.01408311, -0.01408311, -0.0199185 ,  2.47038801, -0.03150329,
        0.        , -0.27808974, -0.04457451, -0.0243975 ])

In [91]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.5891795481569561


In [92]:
results['Prediction'].value_counts()

0    1003
1     679
Name: Prediction, dtype: int64

In [93]:
from sklearn.metrics import f1_score

In [94]:
f1_score(y_test, y_pred, average="weighted")

0.6858982846370403